In [37]:
from ProcessBasic import *

In [69]:
def flatten_directional_traffic_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    將具有 MultiIndex 欄位（方向性交通量）的資料表展平為 long-form。
    自動辨識方向欄位與時間欄位（起、迄）。
    
    Parameters:
        df (pd.DataFrame): 原始的 MultiIndex 欄位資料表
        
    Returns:
        pd.DataFrame: 展平後含方向欄的資料表
    """
    if not isinstance(df.columns, pd.MultiIndex):
        raise ValueError("輸入的欄位不是 MultiIndex 結構")

    # 所有第一層欄位
    all_lv0 = df.columns.get_level_values(0).unique()

    # 自動找出方向欄（排除 起 與 迄）
    direction_cols = [col for col in all_lv0 if col not in ['起', '迄']]

    # 自動找出「起」、「迄」欄位（不假設第二層）
    col_起 = next(col for col in df.columns if col[0] == '起')
    col_迄 = next(col for col in df.columns if col[0] == '迄')

    df_list = []

    for direction in direction_cols:
        temp = df[direction].copy()
        temp['方向'] = direction
        temp['起'] = df[col_起]
        temp['迄'] = df[col_迄]
        df_list.append(temp)

    df_flat = pd.concat(df_list, ignore_index=True)

    # 欄位排序
    ordered_cols = ['方向', '起', '迄'] + [col for col in df_flat.columns if col not in ['方向', '起', '迄']]
    return df_flat[ordered_cols]

def primily_organized(filelist):
    '''讀取交通量檔案檔案清單，並整合成美5分鐘一筆的表格'''
    outputdf = []
    for file in filelist:
        surveynumber = os.path.basename(file).split(".")[0] #讀取檔名進行命名(因xlsx內中並沒有可以協助判斷的部分)
        surveyplace = os.path.basename(file).split(".")[1]

        sheetnamelist = ['平日','假日']
        for sheetname in sheetnamelist:
            df = pd.read_excel(file, header=[0,1], sheet_name=sheetname)
            df.rename(columns = {'Unnamed: 0_level_0':'起'}, inplace=True)
            df.rename(columns = {'Unnamed: 1_level_0':'迄'}, inplace=True)

            # 直接從第二層尋找欄位名稱為 1.8 的位置，在此之前才是調查表格
            drop_from = next(i for i, col in enumerate(df.columns) if col[1] == 1.8)
            df = df.iloc[:, :drop_from] # 使用 iloc 保留 drop_from 之前的所有欄位

            df_flat = flatten_directional_traffic_df(df)
            df_flat['原始資料'] = get_filename_withoutprojectname(file)
            df_flat['調查計畫書點位編號'] = surveynumber
            df_flat['調查路段'] = surveyplace
            df_flat['快慢車道'] = df_flat['方向'].apply(lambda x: '慢車道' if '慢車道' in x else '快車道')
            df_flat['方向'] = df_flat['方向'].str.split('-').str[0]
            df_flat['平假日'] = sheetname
            for column in ['平假日', '方向', '快慢車道', '調查路段', '調查計畫書點位編號']:
                df_flat = move_column(df = df_flat, column_name = column, insert_index=0)
            
            outputdf.append(df_flat)
    
    output = pd.concat(outputdf)

    return output

In [70]:
# Step0 定義資料夾
datafolder = os.path.abspath(os.path.join(get_projectfolderpath(), 'Technical', '06_交通量調查', '02_原始資料'))
volumefolder = os.path.abspath(os.path.join(datafolder, '01_路段交通量'))
speedfolder = os.path.abspath(os.path.join(datafolder, '02_路段旅行速率'))

initialfolder = create_folder(os.path.join(os.getcwd(), '..', '01_資料初步彙整'))
volume_initialfolder = create_folder(os.path.join(initialfolder, '01_路段交通量'))
speed_initialfolder = create_folder(os.path.join(initialfolder, '02_路段旅行速率'))


In [71]:
# Step1 彙整路段交通量資料
files = findfiles(volumefolder,'xlsx')
df = primily_organized(filelist=files)

In [72]:
df

,調查計畫書點位編號,調查路段,快慢車道,方向,平假日,起,迄,聯結車,大貨車,大客車(客運),遊覽車,小型車,機車,自行車&行人,原始資料
0,SL1-1,台61線-後龍觀海大橋,慢車道,往東(北),平日,00:00:00,00:15:00,0,0,0,0,0,2,0,~\台北鼎漢(B_6740)\6787臺鐵海線雙軌化綜合規劃\Technical\06_交通...
1,SL1-1,台61線-後龍觀海大橋,慢車道,往東(北),平日,00:15:00,00:30:00,0,0,0,0,0,1,0,~\台北鼎漢(B_6740)\6787臺鐵海線雙軌化綜合規劃\Technical\06_交通...
2,SL1-1,台61線-後龍觀海大橋,慢車道,往東(北),平日,00:30:00,00:45:00,0,0,0,0,0,1,0,~\台北鼎漢(B_6740)\6787臺鐵海線雙軌化綜合規劃\Technical\06_交通...
3,SL1-1,台61線-後龍觀海大橋,慢車道,往東(北),平日,00:45:00,01:00:00,0,0,0,0,0,2,0,~\台北鼎漢(B_6740)\6787臺鐵海線雙軌化綜合規劃\Technical\06_交通...
4,SL1-1,台61線-後龍觀海大橋,慢車道,往東(北),平日,01:00:00,01:15:00,0,0,0,0,0,0,0,~\台北鼎漢(B_6740)\6787臺鐵海線雙軌化綜合規劃\Technical\06_交通...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,SL4-4,台74線-烏溪橋,快車道,往西(南),假日,22:45:00,23:00:00,0,2,2,0,432,52,0,~\台北鼎漢(B_6740)\6787臺鐵海線雙軌化綜合規劃\Technical\06_交通...
188,SL4-4,台74線-烏溪橋,快車道,往西(南),假日,23:00:00,23:15:00,0,3,2,1,379,37,0,~\台北鼎漢(B_6740)\6787臺鐵海線雙軌化綜合規劃\Technical\06_交通...
189,SL4-4,台74線-烏溪橋,快車道,往西(南),假日,23:15:00,23:30:00,0,4,2,0,310,29,0,~\台北鼎漢(B_6740)\6787臺鐵海線雙軌化綜合規劃\Technical\06_交通...
190,SL4-4,台74線-烏溪橋,快車道,往西(南),假日,23:30:00,23:45:00,0,3,1,0,364,26,0,~\台北鼎漢(B_6740)\6787臺鐵海線雙軌化綜合規劃\Technical\06_交通...


In [45]:
df_flat[df_flat['方向'] == '往西(南)']

,調查計畫書點位編號,調查路段,快慢車道,方向,平假日,起,迄,聯結車,大貨車,大客車(客運),遊覽車,小型車,機車,自行車&行人
96,SL4-4,台74線-烏溪橋,慢車道,往西(南),平日,00:00:00,00:15:00,5,6,0,0,71,17,0
97,SL4-4,台74線-烏溪橋,慢車道,往西(南),平日,00:15:00,00:30:00,4,4,0,1,126,22,0
98,SL4-4,台74線-烏溪橋,慢車道,往西(南),平日,00:30:00,00:45:00,2,5,0,0,109,16,0
99,SL4-4,台74線-烏溪橋,慢車道,往西(南),平日,00:45:00,01:00:00,3,6,0,0,122,21,0
100,SL4-4,台74線-烏溪橋,慢車道,往西(南),平日,01:00:00,01:15:00,4,5,0,0,68,13,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,SL4-4,台74線-烏溪橋,慢車道,往西(南),平日,22:45:00,23:00:00,1,4,1,0,279,33,0
188,SL4-4,台74線-烏溪橋,慢車道,往西(南),平日,23:00:00,23:15:00,3,6,2,1,251,32,0
189,SL4-4,台74線-烏溪橋,慢車道,往西(南),平日,23:15:00,23:30:00,3,4,1,0,219,38,0
190,SL4-4,台74線-烏溪橋,慢車道,往西(南),平日,23:30:00,23:45:00,1,3,1,0,197,28,0


In [46]:
# Step2 彙整路段旅行速率資料